# NEU (Reconfigurations Map and Related Functions)

## **Note:** 
### Run Regression or PCA not this code...this is backend and relies on those to initialize parameters....

## Initialize Modules

###### Imports

In [1]:
# Deep Learning & ML
import tensorflow as tf
import tensorflow_probability as tfp
import keras as K
from keras import backend as Kback
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils

# Linear Regression
from sklearn.linear_model import LinearRegression

# General
import numpy as np
import time

# Alerts
# import tkinter
# from tkinter import messagebox
import os as beepsnd

# General Outputs
print('TensorFlow:', tf.__version__)

Using TensorFlow backend.


TensorFlow: 2.1.0


### Define Reconfiguration Unit

1. Shifts $x \in \mathbb{R}^d$ to $x- c$; c trainable.
2. Rescales componentwise with $a * x$,  $a \in \mathbb{R}^d$ trainable.
3. Applies the map $\psi(x)\triangleq e^{\frac1{1-|x|}}I_{\{|x|<1\}}$ component-wise.  
4. Applies transformation $x \mapsto x +b$, $b \in \mathbb{R}^d$ trainable.
5. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)\mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}.$
6. Applies map $X \mapsto XA$, $A$ is a trainable $d\times d$ matrix.
7. Applies matrix exponential.
8. Multiplies output with result of (1).
9. Re-centers output to $x +c$ where $c$ is as in (1).

In [2]:
class Reconfiguration_unit(K.layers.Layer):
    def __init__(self, *args, **kwargs):
        super(Reconfiguration_unit, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.weight = self.add_weight(shape=[input_shape[1],input_shape[1]], # CURRENTLY NOT A MATRIX FIX THIS!!!
                                    initializer='zeros',
                                    trainable=True)
        self.bias = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.location = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.scale = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        
        
        
    def call(self, x):
        # 1. Shift and scale data
        x_shift = x - self.location
        
        # 2. Rescale componentwise
        x_mod = tf.math.multiply(x_shift,self.scale)
        
        # 3. Apply bumpy function Component-wise
        x_in_abs = tf.math.abs(x_mod)
        Logic_x_leq1 = tf.math.sign(tf.keras.activations.relu(1-x_in_abs)) # Takes value 1 iff |x|<=1 else 0: since probability of |x|=1 is 0 we should be ok
        x_thresheld = Logic_x_leq1*tf.math.exp(-1/(1-tf.math.pow(x_in_abs,-1))) # Computes bump function at thresholds with previous logic
        
        # 4+5. Apply Shift (In Tangent Space) and diagonalize
        x_out = tf.expand_dims((x_in_abs + self.bias), 1) 
        
        # 6. Multiply by weight matrix (in Tangent Space)
        x_out = (x_out * self.weight) 
        
        # 7. Apply Matrix Exponential
        x_out = tf.linalg.expm(x_out)
        
        # 8. Muliply by output of (1)
        x_out = tf.linalg.matvec(x_out,x_shift)
        
        # 9. Recenter Transformed Data
        x_out = x_out + self.location
        
        # Return Ouput
        return x_out

### Basic Algorithm
1. Perform Basic Algorithm (in this case OLS)
2. Map predictions to their graph; ie $x\mapsto (x,\hat{f}_{OLS}(x))$ where $\hat{f}_{OLS}$ is the least-squares regression function.

In [3]:
# Reshape Data Into Compatible Shape
data_x = np.array(data_x).reshape(-1,d)
data_y = np.array(data_y)
# Perform OLS Regression
linear_model = LinearRegression()
reg = linear_model.fit(data_x, data_y)
model_pred_y = linear_model.predict(data_x)
# Map to Graph
data_NEU = np.concatenate((data_x,model_pred_y.reshape(-1,D)),1)

NameError: name 'data_x' is not defined

#### Projection Layer
Maps $\mathbb{X}\left((x,f(x))\mid \theta \right) \in \mathbb{R}^{d\times D}$ to an element of $\mathbb{R}^D$ by post-composing with the second canonical projection
$$
(x_1,x_2)\mapsto x_2
,
$$
where $x_1 \in \mathbb{R}^d$ and $x_2 \in \mathbb{R}^D$.  

In [ ]:
projection_layer = K.layers.Lambda(lambda x: x[:, -D:])

#### Build Reconfiguration

Iniitalize and Build Layers of Reconfiguration

In [6]:
# Initialize NEU
input_layer = K.Input(shape=[(d+D),])
# Build Reconfiguration Map
current_layer = Reconfiguration_unit()
current_layer = current_layer(input_layer)
for i in range(N_Reconfigurations):
    current_layer = Reconfiguration_unit()(current_layer)
# Add Projection Layer
output_layer = projection_layer(current_layer)

NameError: name 'd' is not defined

Compile Model

In [7]:
NEU_OLS = K.Model(inputs=[input_layer], outputs=[output_layer])
NEU_OLS.summary()
print("Model: NEU_OLS Building: - Complete")
beepsnd.system('spd-say "your program has finished"')

NameError: name 'input_layer' is not defined

## Loss Functions

#### NEU-OLS Loss Function (Regression)

In [64]:
def NEU_OLS_loss(y_true,y_predicted):
    SEs = tf.pow(y_true-y_predicted,2)
    out_custom = tfp.stats.percentile(SEs, uncertainty_level)
    return out_custom

In [1]:
def NEU_OLS_entropic(y_true,y_predicted):
    SEs = tf.math.pow(y_true-y_predicted,2)
    out_entropic = K.backend.log(K.backend.sum(K.backend.exp(uncertainty_level*SEs)))/uncertainty_level
    return out_entropic